<a href="https://colab.research.google.com/github/Mtlukasik/Exploration/blob/main/ExplorationDataGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Necessary imports
import numpy as np
import pandas as pd
import string, random
import tensorflow as tf
from google.colab import drive
import json



# Define the constants and helper functions
n_samples = 10000000
n_target_store_ids = 10
n_creative_pack_ids = 100

def lower_string(length, string1=''):
    result = string1+''.join((random.choice(string.ascii_lowercase) for x in range(length)))
    return result

target_store_ids = {i:lower_string(10,'target_') for i in (range(0, n_target_store_ids))}
creative_pack_ids = {i:lower_string(10,'creative_') for i in (range(0, n_creative_pack_ids))}

output = [i for i in range(0, n_target_store_ids)]
target_store_id_probs = {i:j for i,j in enumerate(np.linspace(0, 1, n_target_store_ids+2)[1:-1])}
creative_pack_id_probs = {i:j for i,j in enumerate(np.random.uniform(-0.01, 0.01, n_creative_pack_ids))}

def assign_probability(idc):
    choice = random.choice(output)
    if (target_store_id_probs[choice] + creative_pack_id_probs[idc] >1) or (target_store_id_probs[choice] + creative_pack_id_probs[idc] <0):
      return {(idc,choice) : target_store_id_probs[choice] - creative_pack_id_probs[idc]}
    else:
      return {(idc,choice) : target_store_id_probs[choice] + creative_pack_id_probs[idc]}

creative_mapping2target = [assign_probability(idc) for idc,string in enumerate(creative_pack_ids)]

dataset_cp = []
dataset_tg = []
dataset_label = []

def generate_sample(creative_mapping2target):
  case_chosen = random.choice(creative_mapping2target)
  cp_tg_chosen = list(case_chosen.keys())[0]
  probability = case_chosen[cp_tg_chosen]
  dataset_cp.append(creative_pack_ids[cp_tg_chosen[0]]),dataset_tg.append(target_store_ids[cp_tg_chosen[1]]),dataset_label.append(np.random.binomial(1, probability))

[generate_sample(creative_mapping2target) for i in range(n_samples)]

dataset = pd.DataFrame({"target_store_id": dataset_tg,
                        "creative_pack_id": dataset_cp,
                        "label": dataset_label})

# Mount Google Drive to save the dataset
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
dataset,creative_pack_id_probs, target_store_id_probs

(           target_store_id     creative_pack_id  label
 0        target_vlgqrsanyh  creative_ltpcgpmuwz      1
 1        target_vlgqrsanyh  creative_ltpcgpmuwz      0
 2        target_kfzjpdrrou  creative_sdozbcbgrj      1
 3        target_mymexgkgdu  creative_pcmownnbeg      0
 4        target_vlgqrsanyh  creative_anmneeohsf      0
 ...                    ...                  ...    ...
 9999995  target_itmxppaell  creative_nivolzjgts      0
 9999996  target_kgxkwviasa  creative_yxrqpoxgob      1
 9999997  target_itmxppaell  creative_suapjimvvy      0
 9999998  target_kgxkwviasa  creative_nfiaghnmmm      1
 9999999  target_kgxkwviasa  creative_wtzuiwauei      1
 
 [10000000 rows x 3 columns],
 {0: -0.004746594752955682,
  1: -0.00647024234154066,
  2: 0.0038626658167562125,
  3: -0.002446116457486043,
  4: 0.0011602952655120208,
  5: 0.004349055991519711,
  6: -0.0007564142624434406,
  7: 0.007555090716629886,
  8: -0.007315291822772144,
  9: 0.001250428191841393,
  10: -0.0047758215

In [8]:
import json
# Save the dataset to Google Drive
#data_generation_config.py
# Constants for data generation
N_SAMPLES = 10000000
N_TARGET_STORE_IDS = 10
N_CREATIVE_PACK_IDS = 100

# Paths for saving datasets
DRIVE_PATH = "/content/drive/My Drive/"
PARQUET_FILENAME = "dataset_v{}.parquet"
TFRECORD_FILENAME = "dataset_v{}.tfrecord"
VERSION = 1
METADATA_FILENAME = "metadata_v{}.json"

# Saving Functions
def save_metadata(creative_pack_id_probs, target_store_id_probs):
    metadata = {
        'creative_pack_id_probs': creative_pack_id_probs,
        'target_store_id_probs': target_store_id_probs
    }
    metadata_path = DRIVE_PATH + METADATA_FILENAME.format(VERSION)
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f)
save_metadata(creative_pack_id_probs, target_store_id_probs)
save_path = "/content/drive/My Drive/exploration_syntetic_dataset.csv"
dataset.to_csv(save_path, index=False)
# After dataset generation
# Save the dataset to Google Drive as a parquet file
save_path_parquet = "/content/drive/My Drive/dataset_v1.parquet"
dataset.to_parquet(save_path_parquet, index=False)


# Define the feature description for the TFRecord format
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(target_store_id, creative_pack_id, label):
    feature = {
        'target_store_id': _bytes_feature(target_store_id),
        'creative_pack_id': _bytes_feature(creative_pack_id),
        'label': _int64_feature(label)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

# Write dataset to TFRecord
with tf.io.TFRecordWriter("/content/drive/My Drive/dataset_v1.tfrecord") as writer:
    for _, row in dataset.iterrows():
        example = serialize_example(row["target_store_id"], row["creative_pack_id"], row["label"])
        writer.write(example)
